In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from collections import defaultdict

from model.Base import Base
from model.Device import Device
from model.DeviceAppTraffic import DeviceAppTraffic
from model.DnsReq import DnsReq
from model.Flow import Flow
from model.User import User
from model.user_devices import user_devices

from sqlalchemy import create_engine, text, func
from sqlalchemy.orm import sessionmaker
from sqlalchemy.pool import NullPool

from IPython.display import display

#output_notebook()

DB='postgresql+psycopg2:///ucnstudy'

engine = create_engine(DB, echo=False, poolclass=NullPool)
Base.metadata.bind = engine
Session = sessionmaker(bind=engine) 

Analyze device usage per user

In [2]:
ses = Session()

users = ses.query(User)

for user in users:
    sql_user_devices = text('select * from user, user_devices where user_devices.user_id =:user').bindparams(user = user.id)
    user_devices = ses.execute(sql_user_devices)

    print ("user: " + str(user.id))
    for devices in user_devices:
        sql_domains_accessed = text('select dnsreqs.query_domain, count(*) from dnsreqs, user_devices where dnsreqs.devid =:devid group by dnsreqs.query_domain').bindparams(devid = devices.device_id)
        domains_accessed = ses.execute(sql_domains_accessed)
    print ("device: " + str(devices.device_id))
        dom = []
        accesses = []
        length = 0
        for domains in domains_accessed:
            #print (domains.query_domain, domains.count)
            dom.append(domains.query_domain)
            accesses.append(domains.count)
            length = length + 1
        x = np.arange(length)
        plt.bar(x, accesses)
        plt.xticks(x + 0.5, dom, rotation = 90)
        plt.show()


IndentationError: unexpected indent (<ipython-input-2-4f870915f7f6>, line 14)

Analizing beginning and end of daily activities 

In [6]:
ses = Session()

devices = ses.query(Device).distinct().order_by(Device.id)

for dev in devices: 
    
    sql_beg_day = text("SELECT devid, startts FROM flows join (SELECT DATE(startts) as date_entere\
d, MIN(startts) as min_time FROM flows WHERE devid =:d_id GROUP BY date(startts)) AS grp ON grp.min_ti\
me = flows.startts order by flows.startts limit 400;").bindparams(d_id = dev.id)
    result_beg_day = ses.execute(sql_beg_day)
    
    info = defaultdict(list)
    for row in result_beg_day:
        info['devid'].append(str(row[0]))
        info['start'].append(row[1])
        
    sql_end_day = text("SELECT devid, endts FROM flows join (SELECT DATE(endts) as date_entered, M\
AX(endts) as max_time FROM flows WHERE devid =:d_id GROUP BY date(endts)) AS grp ON grp.max_time = flo\
ws.endts order by flows.endts limit 400;").bindparams(d_id = dev.id)
    result_end_day = ses.execute(sql_end_day)

    for row in result_end_day:
        info['end'].append(row[1])

    table = pd.DataFrame(info)
    display(table)

ses.close()

,devid,end,start
0,1,2015-03-05 13:47:02.622617,2015-03-05 10:31:16.214518
1,1,2015-03-06 16:39:06.818626,2015-03-06 14:22:11.114880
2,1,2015-03-07 23:49:01.854332,2015-03-07 10:18:58.578948
3,1,2015-03-17 23:45:08.831911,2015-03-17 20:12:04.855609
4,1,2015-03-19 23:59:57.428200,2015-03-19 01:34:56.387976
5,1,2015-03-20 00:15:32.173448,2015-03-20 00:01:01.113738
6,1,2015-03-21 23:59:46.475630,2015-03-21 08:13:44.100283
7,1,2015-03-22 00:16:23.540436,2015-03-22 00:00:15.879419
8,1,2015-03-25 00:17:01.372017,2015-03-25 00:01:57.562157
9,1,2015-03-28 22:04:24.233281,2015-03-28 22:04:24.120179


""


,devid,end,start
0,3,2015-04-18 14:55:16.534610,2015-04-18 14:24:29.663117
1,3,2015-04-19 14:46:22.325818,2015-04-19 02:37:08.263501
2,3,2015-04-20 22:21:36.330138,2015-04-20 03:06:35.546421
3,3,2015-04-24 19:32:47.087257,2015-04-24 19:32:46.849299
4,3,2015-04-27 19:27:22.470298,2015-04-27 19:14:16.797616
5,3,2015-04-28 17:35:57.809929,2015-04-28 17:35:57.778453
6,3,2015-05-02 23:57:19.872874,2015-05-02 22:33:54.103909
7,3,2015-05-03 23:43:24.536313,2015-05-03 00:08:44.129625
8,3,2015-05-04 23:33:21.959675,2015-05-04 15:09:12.355742
9,3,2015-05-05 04:01:13.132237,2015-05-05 04:01:12.956264


,devid,end,start
0,4,2015-03-01 21:00:18.975444,2015-03-01 07:47:31.058475
1,4,2015-03-03 09:55:16.404654,2015-03-03 06:49:27.868855
2,4,2015-03-05 23:19:28.926246,2015-03-05 00:18:02.356548
3,4,2015-03-06 23:59:43.137605,2015-03-06 09:27:49.032141
4,4,2015-03-07 16:02:46.569310,2015-03-07 00:01:03.762147
5,4,2015-03-08 00:16:53.475028,2015-03-08 00:00:23.252136
6,4,2015-03-09 21:52:18.265992,2015-03-09 07:54:28.302966
7,4,2015-03-12 21:35:21.700963,2015-03-12 00:27:59.900232
8,4,2015-03-13 09:25:09.694741,2015-03-13 09:08:49.377171
9,4,2015-03-14 23:59:39.290760,2015-03-14 23:02:44.555990


,devid,end,start
0,5,2015-02-26 20:08:43.609554,2015-02-26 17:07:27.365762
1,5,2015-03-01 23:57:37.499950,2015-03-01 12:05:36.719980
2,5,2015-03-02 00:16:58.711643,2015-03-02 00:00:24.089288
3,5,2015-03-05 22:21:58.679227,2015-03-05 13:29:47.403232
4,5,2015-03-09 23:52:49.112035,2015-03-09 13:52:42.413849
5,5,2015-03-12 23:59:00.320442,2015-03-12 00:19:28.247003
6,5,2015-03-13 21:51:56.021490,2015-03-13 00:04:29.538558
7,5,2015-03-21 13:34:01.366030,2015-03-21 08:06:10.479144
8,5,2015-03-25 22:50:31.663430,2015-03-25 22:50:27.623807
9,5,2015-03-29 23:47:58.533565,2015-03-29 00:18:37.448983


...

""


,devid,end,start
0,16,2015-05-10 13:27:49.970496,2015-05-10 13:27:49.840017


,devid,end,start
0,17,2015-04-20 22:21:36.363378,2015-04-20 22:21:36.117999
1,17,2015-04-24 22:06:04.085702,2015-04-24 22:06:03.835147
2,17,2015-05-04 13:57:42.691187,2015-05-04 13:51:03.721921
3,17,2015-05-11 13:46:29.610165,2015-05-11 13:46:29.378027
4,17,2015-06-15 03:22:28.213818,2015-06-15 03:04:43.074144


,devid,end,start
0,18,2015-11-10 14:58:47.202157,2015-11-10 10:10:23.070493
1,18,2015-11-18 09:50:38.250751,2015-11-18 09:41:01.357190
2,18,2015-11-22 16:38:54.390309,2015-11-22 16:36:05.332247
3,18,2015-11-23 09:48:54.854906,2015-11-23 09:48:52.035277
4,18,2015-11-25 13:34:42.623733,2015-11-25 09:44:45.026419


ValueError: arrays must all be same length